# NRPy+'s Reference Metric Interface

### NRPy+ Source Code for this module: [reference_metric.py](../edit/reference_metric.py), [reference_metric__hatted_quantities.py](../edit/reference_metric__hatted_quantities.py)

### Why use a reference metric? Benefits of choosing the best coordinate system for the problem

When solving a partial differential equation on the computer, it is useful to first pick a coordinate system well-suited to the physical scenario we are modeling. 

For example, if we are modeling a spherically-symmetric star, it would be hugely wasteful to model the star in 3-dimensional Cartesian coordinates ($x$,$y$,$z$). This is because in Cartesian coordinates, we would need to choose high sampling in all three Cartesian directions. If instead we chose to model the star in spherical coordinates ($r$,$\theta$,$\phi$), so long as the star is centered at $r=0$, we would not need to model the star in more than one point in the $\theta$ and $\phi$ directions!

The same argument holds for stars that vary slowly in $\theta$ and $\phi$ directions (like isolated neutron stars or black holes), in which case the number of points needed to sample the angular directions will still be much smaller than in the radial direction.

Thus choice of an appropriate reference metric may directly mitigate the [Curse of Dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality).

### *reference_metric.py*: Defining a reference metric

***Note that currently only orthogonal reference metrics of dimension 3 or fewer are supported. This can be extended if desired.***

NRPy+ assumes all curvilinear coordinate systems map directly from a uniform, Cartesian numerical grid with coordinates $(x,y,z)$=(xx\[0\],xx\[1\],xx\[2\]). Thus when defining reference metrics, all defined coordinate quantities must be in terms of the xx\[\] array. As we will see, this adds a great deal of flexibility

For example, **reference_metric.py** requires that the *orthogonal coordinate scale factors* be defined. As described [here](https://en.wikipedia.org/wiki/Curvilinear_coordinates), the $i$th scale factor is the positive root of the metric element $g_{ii}$. In ordinary spherical coordinates $(r,\theta,\phi)$, with line element $ds^2 = g_{ij} dx^i dx^j = dr^2+ r^2 d \theta^2 + r^2 \sin^2\theta \ d\phi^2$, we would first define
* $r = xx_0$
* $\theta = xx_1$
* $\phi = xx_2$,

so that the scale factors are defined as
* scalefactor_orthog[0] = $1$
* scalefactor_orthog[1] = $r$
* scalefactor_orthog[2] = $r \sin \theta$

Here is the corresponding code:

In [1]:
import sympy as sp
import NRPy_param_funcs as par
import reference_metric as rfm

r = rfm.xx[0]
th = rfm.xx[1]
ph = rfm.xx[2]

rfm.scalefactor_orthog[0] = 1
rfm.scalefactor_orthog[1] = r
rfm.scalefactor_orthog[2] = r*sp.sin(th)

# Notice that the scale factor will be given 
#    in terms of the fundamental Cartesian
#    grid variables, and not {r,th,ph}:
print("r*sin(th) = "+str(rfm.scalefactor_orthog[2]))

r*sin(th) = xx0*sin(xx1)


Next suppose we wish to modify our radial coordinate $r(xx_0)$ to be an exponentially increasing function, so that our numerical grid $(xx_0,xx_1,xx_2)$ will map to a spherical grid with radial grid spacing ($\Delta r$) that *increases* with $r$. Generally we will find it useful to define $r(xx_0)$ to be an odd function, so let's choose

$$r(xx_0) = a \sinh(xx_0/s),$$

where $a$ is an overall radial scaling factor, and $s$ denotes the scale (in units of $xx_0$ over which exponential growth will take place. In our implementation below, note that we use the relation

$$\sinh(x) = \frac{e^x - e^{-x}}{2},$$

as SymPy finds it easier to evaluate exponentials than hyperbolic trigonometric functions.

In [2]:
a,s = sp.symbols('a s',positive=True)
xx0_rescaled = rfm.xx[0] / s
r = a*(sp.exp(xx0_rescaled) - sp.exp(-xx0_rescaled))/2

# Must redefine the scalefactors since 'r' has been updated!
rfm.scalefactor_orthog[0] = 1
rfm.scalefactor_orthog[1] = r
rfm.scalefactor_orthog[2] = r*sp.sin(th)

print(rfm.scalefactor_orthog[2])

a*(exp(xx0/s) - exp(-xx0/s))*sin(xx1)/2


Often we will find it useful to also define the appropriate mappings from (xx\[0\],xx\[1\],xx\[2\]) to Cartesian coordinates (for plotting purposes) and ordinary spherical coordinates (e.g., in case initial data when solving a PDE are naturally written in spherical coordinates). For this purpose, reference_metric.py also declares lists **xxCart\[\]** and **xxSph\[\]**, which in this case are defined as

In [3]:
rfm.xxSph[0] = r
rfm.xxSph[1] = th
rfm.xxSph[2] = ph

rfm.xxCart[0] = r*sp.sin(th)*sp.cos(ph)
rfm.xxCart[1] = r*sp.sin(th)*sp.sin(ph)
rfm.xxCart[2] = r*sp.cos(th)

# Here we show off SymPy's pretty_print() 
#   and simplify() functions. Nice, no?
sp.pretty_print(sp.simplify(rfm.xxCart[0]))

                        ⎛xx₀⎞
a⋅sin(xx₁)⋅cos(xx₂)⋅sinh⎜───⎟
                        ⎝ s ⎠


### ***reference_metric__hatted_quantities.py***: Computing basic geometric quantities

Based on the quantities defined in reference_metric.py (actually just the orthogonal scale factors scalefactor_orthog\[\]), **reference_metric__hatted_quantities.py** evaluates a number of geometric quantities useful for solving PDEs in curvilinear coordinate systems. We call geometric quantities related to the reference metric "hatted" quantities, adopting the nomenclature of Baumgarte (2012). For example, the reference metric itself is defined as $\hat{g}_{ij}$=ghatDD\[i\]\[j\]:

In [4]:
import reference_metric__hatted_quantities as rhq

rhq.reference_metric__hatted_quantities()

sp.pretty_print(sp.Matrix(sp.simplify(rhq.ghatDD)))

⎡1           0                         0              ⎤
⎢                                                     ⎥
⎢                     2                               ⎥
⎢      ⎛ xx₀    -xx₀ ⎞                                ⎥
⎢      ⎜ ───    ─────⎟                                ⎥
⎢    2 ⎜  s       s  ⎟                                ⎥
⎢   a ⋅⎝ℯ    - ℯ     ⎠                                ⎥
⎢0  ───────────────────                0              ⎥
⎢            4                                        ⎥
⎢                                                     ⎥
⎢                                          2          ⎥
⎢                           ⎛ xx₀    -xx₀ ⎞           ⎥
⎢                           ⎜ ───    ─────⎟           ⎥
⎢                         2 ⎜  s       s  ⎟     2     ⎥
⎢                        a ⋅⎝ℯ    - ℯ     ⎠ ⋅sin (xx₁)⎥
⎢0           0           ─────────────────────────────⎥
⎣                                      4              ⎦


In addition to $\hat{g}_{ij}$, **reference_metric__hatted_quantities.py** also provides:
* The rescaling "matrix" ReDD\[i\]\[j\], used for separating singular (due to chosen coordinate system) pieces of smooth rank-2 tensor components from the smooth parts, so that the smooth parts can be used within temporal and spatial differential operators.
* First and second derivatives of the reference metric: $\hat{g}_{ij,k}$=ghatDD_dD\[i\]\[j\]\[k\]; $\hat{g}_{ij,kl}$=ghatDD_dDD\[i\]\[j\]\[k\]\[l\]
* Christoffel symbols associated with the reference metric, $\hat{\Gamma}^i_{jk}$ = GammahatUDD\[i\]\[j\]\[k\] and their first derivatives $\hat{\Gamma}^i_{jk,l}$ = GammahatUDD_dD\[i\]\[j\]\[k\]\[l\]
